# CSV + API

In this reboot, we are going to use:

- The [Goodreads books](https://www.kaggle.com/jealousleopard/goodreadsbooks) dataset from Kaggle.
- The [Open Library Books API](https://openlibrary.org/dev/docs/api/books)

The goal of this livecode is to load the data from a CSV + loop over rows to enrich each row with information such as:

- List of subjects (Science, Humor, Travel, etc.)
- The cover URL of the book
- Other information you'd find useful in the JSON API

First, download the CSV in the local folder:

In [1]:
!curl -L https://gist.githubusercontent.com/ssaunier/351b17f5a7a009808b60aeacd1f4a036/raw/books.csv > books.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1509k  100 1509k    0     0   216k      0  0:00:06  0:00:06 --:--:--  357k


In [2]:
!ls -lh

total 1.5M
-rw-r--r-- 1 ataka ataka 1.5M Dec 18 21:03 books.csv
-rw-r--r-- 1 ataka ataka  651 Dec 18 21:02 README.md
-rw-r--r-- 1 ataka ataka 2.8K Dec 18 21:02 Recap.ipynb


Then import the usual suspects!

In [4]:

import numpy as np
import pandas as pd
import requests

# Pandas tablolarını düzgün görmek için ayar
pd.set_option('display.max_columns', None)

## Load books from CSV

In [6]:
books_df = pd.read_csv('books.csv', on_bad_lines='skip')

books_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964


Let's add a new column

In [7]:
books_df['cover_url'] = None
books_df['subjects'] = None

In [8]:
# Harry Potter'ın ISBN'i
isbn = "9780439785969" 

# API URL'ini oluşturuyoruz
url = f"https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json&jscmd=data"

# İsteği gönder
response = requests.get(url).json()

# Gelen cevabı görelim
response

{'ISBN:9780439785969': {'url': 'https://openlibrary.org/books/OL24280830M/Harry_Potter_and_the_Half-Blood_Prince',
  'key': '/books/OL24280830M',
  'title': 'Harry Potter and the Half-Blood Prince',
  'authors': [{'url': 'https://openlibrary.org/authors/OL23919A/J.K._Rowling',
    'name': 'J.K. Rowling'},
   {'url': 'https://openlibrary.org/authors/OL2703953A/Mary_GrandPré',
    'name': 'Mary GrandPré'}],
  'number_of_pages': 652,
  'pagination': '652p',
  'identifiers': {'amazon': ['0439785960'],
   'goodreads': ['53178655'],
   'isbn_10': ['0439785960'],
   'isbn_13': ['9780439785969'],
   'oclc': ['70666878', '819153929'],
   'openlibrary': ['OL24280830M']},
  'classifications': {'lc_classifications': ['PZ7.R79835Halc 2005']},
  'publishers': [{'name': 'Scholastic'}],
  'publish_places': [{'name': 'New York, USA'}],
  'publish_date': '2006-09',
  'subjects': [{'name': 'orphans',
    'url': 'https://openlibrary.org/subjects/orphans'},
   {'name': 'foster homes',
    'url': 'https://o

## API - Open Library

In [10]:
def fetch_book_info(isbn):
   
    url = f"https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json&jscmd=data"
    
    try:
        response = requests.get(url).json()
        
        # API bazen boş dönebilir, kontrol edelim
        key = f"ISBN:{isbn}"
        if key in response:
            book_data = response[key]
            
            cover_url = book_data.get("cover", {}).get("medium", "")
            
            subjects = book_data.get("subjects", [])
            subject_names = ", ".join([s["name"] for s in subjects])
            
            return cover_url, subject_names
            
    except:
        pass 
    
    return "", "" 

# Test edelim
fetch_book_info("9780439785969")

('https://covers.openlibrary.org/b/id/14860369-M.jpg',
 "orphans, foster homes, romans, magie, adolescence, Quill Award winner, Scottish Children’s Book Award winner, British Book of the Year Award winner, Fiction, Juvenile fiction, Magic, Schools, Witches, Wizards, New York Times bestseller, Fantasy fiction, nyt:series_books=2006-07-15, nyt:series_books=2006-09-16, Romans, nouvelles, etc. pour la jeunesse, Sorciers, Roman fantastique, Merveilleux, Hogwarts School of Witchcraft and Wizardry (Imaginary place), Harry Potter (Fictitious character), Hogwarts School of Witchcraft and Wizardry (Imaginary organization), Magos, Magia, Ficción juvenil, Escuelas, Novela fantástica, England, School stories, Family, Harry Potter (Fictional character), Orphans & Foster Homes, Social Themes, Fantasy, Fantasy & Magic, Friendship, Reading Level-Grade 11, Reading Level-Grade 10, Reading Level-Grade 12, England, fiction, Magic, fiction, Schools, fiction, Children's fiction, Potter, harry (fictitious cha

## Calling the API with multiple ISBNs at a time

In [11]:
# Sadece ilk 20 kitabı alalım (Kopyasını oluşturuyoruz)
small_df = books_df.head(20).copy()

# Yeni boş sütunlar açalım
small_df['cover_url'] = ""
small_df['subjects'] = ""

print("Veriler çekiliyor, lütfen bekleyin...")

# Her satır için döngü (iterrows)
for index, row in small_df.iterrows():
    isbn = row['isbn13'] # ISBN numarasını al
    
    # Fonksiyonumuzu çağırıp bilgileri alalım
    cover, subjects = fetch_book_info(isbn)
    
    # Tabloya yazalım
    small_df.at[index, 'cover_url'] = cover
    small_df.at[index, 'subjects'] = subjects
    
    print(f"Kitap {index+1} tamamlandı.")

# Sonuca bakalım
small_df[['title', 'cover_url', 'subjects']].head()

Veriler çekiliyor, lütfen bekleyin...
Kitap 1 tamamlandı.
Kitap 2 tamamlandı.
Kitap 3 tamamlandı.
Kitap 4 tamamlandı.
Kitap 5 tamamlandı.
Kitap 6 tamamlandı.
Kitap 7 tamamlandı.
Kitap 8 tamamlandı.
Kitap 9 tamamlandı.
Kitap 10 tamamlandı.
Kitap 11 tamamlandı.
Kitap 12 tamamlandı.
Kitap 13 tamamlandı.
Kitap 14 tamamlandı.
Kitap 15 tamamlandı.
Kitap 16 tamamlandı.
Kitap 17 tamamlandı.
Kitap 18 tamamlandı.
Kitap 19 tamamlandı.
Kitap 20 tamamlandı.


,title,cover_url,subjects
0,Harry Potter and the Half-Blood Prince (Harry ...,https://covers.openlibrary.org/b/id/14860369-M...,"orphans, foster homes, romans, magie, adolesce..."
1,Harry Potter and the Order of the Phoenix (Har...,https://covers.openlibrary.org/b/id/14656833-M...,"Children's Books/Ages 9-12 Fiction, Witches an..."
2,Harry Potter and the Sorcerer's Stone (Harry P...,https://covers.openlibrary.org/b/id/7572543-M.jpg,"series:Harry_Potter, Ghosts, Monsters, Vampire..."
3,Harry Potter and the Chamber of Secrets (Harry...,https://covers.openlibrary.org/b/id/10301720-M...,"series:Harry_Potter, Fantasy fiction, school s..."
4,Harry Potter and the Prisoner of Azkaban (Harr...,https://covers.openlibrary.org/b/id/8778528-M.jpg,"Fantasy fiction, orphans, foster homes, fantas..."


In [12]:
# Dosyayı CSV olarak kaydet
small_df.to_csv("enriched_books.csv", index=False)
print("Dosya başarıyla kaydedildi!")

Dosya başarıyla kaydedildi!


In [13]:
small_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count,cover_url,subjects
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249,https://covers.openlibrary.org/b/id/14860369-M...,"orphans, foster homes, romans, magie, adolesce..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613,https://covers.openlibrary.org/b/id/14656833-M...,"Children's Books/Ages 9-12 Fiction, Witches an..."
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390,https://covers.openlibrary.org/b/id/7572543-M.jpg,"series:Harry_Potter, Ghosts, Monsters, Vampire..."
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272,https://covers.openlibrary.org/b/id/10301720-M...,"series:Harry_Potter, Fantasy fiction, school s..."
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964,https://covers.openlibrary.org/b/id/8778528-M.jpg,"Fantasy fiction, orphans, foster homes, fantas..."
